In [1]:
import time
import spacy
import lftk
import pandas as pd
from datasets import load_dataset


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/homebrew/Caskroom/miniforge/base/envs/replicate_numpy_error/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/homebrew/Caskroom/miniforge/base/envs/replicate_numpy_error/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/replicate_numpy_error/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/C

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/homebrew/Caskroom/miniforge/base/envs/replicate_numpy_error/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/homebrew/Caskroom/miniforge/base/envs/replicate_numpy_error/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/replicate_numpy_error/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/C

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/homebrew/Caskroom/miniforge/base/envs/replicate_numpy_error/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/homebrew/Caskroom/miniforge/base/envs/replicate_numpy_error/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/replicate_numpy_error/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/C

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [4]:
# load a trained pipeline of your choice from spacy
nlp = spacy.load("en_core_web_sm")

# LFTK feature families of interest
readformula = lftk.search_features(family="readformula", return_format = "list_key")
wordsent = lftk.search_features(family="wordsent", return_format = "list_key")
worddiff = lftk.search_features(family="worddiff", return_format = "list_key")

target_features = readformula + wordsent + worddiff
readformula
wordsent
# worddiff

['t_word',
 't_stopword',
 't_punct',
 't_syll',
 't_syll2',
 't_syll3',
 't_uword',
 't_sent',
 't_char']

In [3]:
# data: generated summaries, gold: human-written summaries
gen = pd.read_csv('../output/deliverable_2/pegasusbillsum_baseline.csv', usecols=['summary_generated'])
gold = load_dataset("FiscalNote/billsum")["test"].to_pandas()
gold.drop(columns=['text', 'title'], inplace=True)
gen.rename(columns={"summary_generated": "summary"}, inplace=True)

Using custom data configuration FiscalNote--billsum-7c662dd9fff22026
Found cached dataset parquet (/Users/anantha/.cache/huggingface/datasets/FiscalNote___parquet/FiscalNote--billsum-7c662dd9fff22026/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# return a list of dictionaries; each dictionary has all features with values for the summary at index I
def lftk_eval(df):
    start_time = time.time()
    docs = [nlp(s) for s in df.summary.tolist()]
    LFTK = lftk.Extractor(docs = docs)
    extracted = LFTK.extract(features = target_features)
    print(f"Generated summary features extracted in {time.time() - start_time} sec")
    return extracted

In [ ]:
# Log runtime for adding all these linguistic features
print("generated summaries")
gen_lftk = lftk_eval(gen)
print("gold summaries")
gold_lftk = lftk_eval(gold)

generated summaries


In [ ]:
# Merge summary columns on corresponding features
def add_features(df, fdicts):
    fdf = pd.DataFrame(fdicts)
    new = df.merge(fdf, left_index=True, right_index=True)
    return new

In [22]:
gen_final = add_features(gen, gen_lftk)
gold_final = add_features(gold, gold_lftk)

In [12]:
# Write metrics for each summary to output
MODEL_NAME = "pegasusbillsum"
gen_final.to_csv(MODEL_NAME + "_baseline_lftk.csv")
gold_final.to_csv("gold_lftk.csv")

In [24]:
# Write basic stat report as well
# gen.describe().to_csv("report_gen_lftk.csv")
gen_final.describe()
# gold_final.describe().to_csv("report_gold_lftk.csv")
# gold_final.to_csv("gold_lftk.csv")

,fkre,fkgl,fogi,smog,cole,auto,t_word,t_stopword,t_punct,t_syll,t_syll2,t_syll3,t_uword,t_sent,t_char,t_kup,t_bry,t_subtlex_us_zipf
count,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000,3269.000000
mean,5.292479,26.011575,32.256271,12.407292,14.809797,30.819341,140.939125,58.409299,19.563169,242.309575,36.700214,13.804833,81.702967,3.140410,744.964209,725.648951,581.660572,694.295961
std,39.147077,14.147657,14.545976,4.721686,2.464433,17.997384,67.205049,31.593493,11.750572,115.167318,18.685703,8.700924,29.539885,1.943082,350.606323,379.995074,307.658775,339.627031
min,-197.615000,6.217000,8.969000,0.000000,6.261000,6.407000,25.000000,5.000000,1.000000,35.000000,3.000000,0.000000,18.000000,1.000000,110.000000,44.990000,31.536000,102.607000
25%,-8.085000,17.666000,23.667000,9.329000,13.118000,20.248000,78.000000,30.000000,10.000000,136.000000,21.000000,7.000000,56.000000,2.000000,419.000000,378.490000,302.351000,384.765000
50%,12.846000,22.472000,28.619000,11.892000,14.684000,26.103000,140.000000,56.000000,18.000000,242.000000,35.000000,12.000000,84.000000,3.000000,745.000000,707.880000,561.962000,683.621000
75%,29.003000,29.742000,36.000000,14.750000,16.320000,35.103000,213.000000,87.000000,28.000000,352.000000,52.000000,19.000000,107.000000,4.000000,1094.000000,1081.000000,869.513000,1035.688000
max,81.956000,97.404000,105.737000,36.579000,24.759000,122.723000,251.000000,135.000000,85.000000,509.000000,98.000000,64.000000,148.000000,22.000000,1499.000000,1517.120000,1274.352000,1317.443000
